## Se va a crear un CSV con las etiquetas descargadas de Open Image v6
Cuando se descargó las imágenes se generó una carpeta paor cada clase distinta con las imágenes y dentro de cada clase encontramos otra carpeta llamada labels donde se tiene cada un txt por cada imagen con las coordenadas del bounding box en el formato xmin, ymin, xmax e y max. Obtendremos la información de los txt y los pasaremos a un Dataframe con el formato x, y, heigth y weigth, para posteriormente utilizarlos en los Transformers DETR.
  
Con la descarga de las imágenes ya se descarga un CSV con la información pero es demasiado grande para usar ya que están TODAS las distintas clases que tiene Open Image v6, es mas nisiquiera se puede abrir con algunos editores de texto.

In [1]:
import pandas as pd
import numpy as np
import os
import re
from PIL import Image

label_path = "labels" #Nombre de la carpeta con las etiquetas
classes_names = ["mouse", "red_panda"] #nombre de las clases y de las carpetas donde se encuentran las imágenes
path_to_classes = "TransformerDataset\\train"
path = os.getcwd()

In [2]:
#Obtenemos los nombres de los txt, que es el mismo que el de las imágenes
txt_names = [f for f in os.listdir(os.path.join(path,*[path_to_classes,classes_names[0],label_path])) 
               if os.path.isfile(os.path.join(path,*[path_to_classes,classes_names[0],label_path,f])) 
                                     and f.endswith(".txt")]
print(len(txt_names))

300


In [3]:
#Obtenemos una lista de todas las imágenes para comprobar si existe el archivo label correspondiente
img_list = [f for f in os.listdir(os.path.join(path,*[path_to_classes,classes_names[0]])) 
               if os.path.isfile(os.path.join(path,*[path_to_classes,classes_names[0],f])) 
                                     and f.endswith(".jpg")]

txt_names = [f for f in os.listdir(os.path.join(path,*[path_to_classes,classes_names[0],label_path])) 
               if os.path.isfile(os.path.join(path,*[path_to_classes,classes_names[0],label_path,f])) 
                                     and f.endswith(".txt") and f[:-4]+".jpg" in img_list]

print(len(img_list))
print(len(txt_names))

300
300


Diccionario de lista que almacena los datos para luego generar el DataFrame con los etiquetados.

In [4]:
#Diccionario de vectores para el DataFrame
dictionary = {'image_name':[], 
           'x':[],
           'y':[],
           'width':[],
           'height':[],
           'class':[] } 

Hay que hacer el texto a continuación por cada clase distinta que tengamos en classes_names. 

In [5]:
pattern = re.compile("([a-zA-Z_]*) (\d*.\d*) (\d*.\d*) (\d*.\d*) (\d*.\d*)")

for name in txt_names:
    #Abrimos cada txt
    for i, line in enumerate(open(os.path.join(path,*[path_to_classes,classes_names[0],label_path,name]))):
        match  = re.search(pattern,line)
        if match:
            dictionary["image_name"].append(name[0:-4]+".jpg")
            dictionary["class"].append(match.group(1))
            xmin = float(match.group(2))
            ymin = float(match.group(3))
            xmax = float(match.group(4))
            ymax = float(match.group(5))    
            
            #El tamaño del bounding box se calcula de la siguiente manera:
            bb_width = xmax-xmin
            bb_height = ymax-ymin
            dictionary["width"].append(bb_width)
            dictionary["height"].append(bb_height)
            
            #Los valores x e y que son el centro del rectángulo se calculan de la siguiente manera:
            x_center = xmin + bb_width/2
            y_center = ymin + bb_height/2
            dictionary["x"].append(x_center)
            dictionary["y"].append(y_center)
 

In [6]:
#Comprobamos los length
print(len(dictionary["image_name"]))
print(len(dictionary["class"]))
print(len(dictionary["x"]))
print(len(dictionary["y"]))
print(len(dictionary["width"]))
print(len(dictionary["height"]))

372
372
372
372
372
372


In [7]:
df = pd.DataFrame(dictionary) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   image_name  372 non-null    object 
 1   x           372 non-null    float64
 2   y           372 non-null    float64
 3   width       372 non-null    float64
 4   height      372 non-null    float64
 5   class       372 non-null    object 
dtypes: float64(4), object(2)
memory usage: 17.6+ KB


In [8]:
df.to_csv(os.path.join(path,*[path_to_classes,"train_mouse_red_panda.csv"]), index=False)

## Visualización de DataFrames

In [10]:
df = pd.read_csv(os.path.join(path,*[path_to_classes,"train_mouse_red_panda.csv"]))
df.head()

,image_name,x,y,width,height,class
0,00165018e603855f.jpg,505.280000,363.691200,366.720000,212.580240,mouse
1,001d91060bb76e5e.jpg,524.399616,410.799900,912.799744,818.399400,mouse
2,001e1cf3abbee3cd.jpg,512.960000,287.072198,832.640000,512.628655,mouse
3,002c5a9026f5415c.jpg,499.520000,352.061912,188.800000,218.917892,mouse
4,003de0d25a90a01a.jpg,582.808576,336.810000,878.024704,630.020436,mouse
